In [10]:
using Images
using FileIO
using DynamicPolynomials

@polyvar x
@polyvar y
@polyvar z
@polyvar w



(w,)

In [12]:

function progs(ndims, order)
    m = 1
    dims = ["x", "y", "z", "w"]

    progs = Dict{String, Vector{String}}()
    for i in 1:ndims
        d = dims[i]
        progs[d] = Vector{String}()
        push!(progs[d], "a[$m]")
        m += 1
        for i1 in 1:ndims
            push!(progs[d], "a[$m] * $(dims[i1])")
            m += 1
            for i2 in i1:ndims
                push!(progs[d], "a[$m] * $(dims[i1]) * $(dims[i2])")
                m += 1
                if order > 2
                    for i3 in i2:ndims
                        push!(progs[d], "a[$m] * $(dims[i1]) * $(dims[i2]) * $(dims[i3])")
                        m += 1
                        if order > 3
                            for i4 in i3:ndims
                                push!(progs[d], "a[$m] * $(dims[i1]) * $(dims[i2]) * $(dims[i3]) * $(dims[i4])")
                                m += 1
                                if order > 4
                                    for i5 in i4:ndims
                                        push!(progs[d], "a[$m] * $(dims[i1]) * $(dims[i2]) * $(dims[i3]) * $(dims[i4]) * $(dims[i5])")
                                        m += 1
                                    end
                                end
                            end
                        end
                    end
                end
            end
        end
    end
    progs, m-1
end


progs (generic function with 1 method)

In [15]:
function polys(es::Dict{String, Expr}, a::Vector{Float64})
    ps = Dict{String, DynamicPolynomials.Polynomial{true,Float64}}()

    for d in keys(es)
        ps[d] = eval(es[d])
    end
    ps
end

polys (generic function with 1 method)

In [31]:

function exprs(ps::Dict{String, Vector{String}}, as::Int64)

    es = Dict{String, Expr}()
    for d in keys(ps)
        es[d] = Meta.parse(join(ps[d], " + "))
    end

    es
end


exprs (generic function with 3 methods)

In [16]:


function string2args(s::String)
    [(Int(c)-77)/10.0 for c in s]
end


string2args (generic function with 1 method)

In [43]:


function write_raw(iters, w, h)
    hrange = (minimum(iters["x"]), maximum(iters["x"]))
    hscale = (w-1) / (hrange[2]-hrange[1])
    yrange = (minimum(iters["y"]), maximum(iters["y"]))
    yscale = (h-1) / (yrange[2]-yrange[1])

    pixels = zeros(Int64, w, h)

    for i in 1:length(iters["x"])
        x = floor(Int64, 1+(iters["x"][i] - hrange[1]) * hscale)
        y = floor(Int64, 1+(iters["y"][i] - yrange[1]) * yscale)
        pixels[x, y] += 1
    end

    gmax = 1 / maximum(pixels)
    
    gs =  (pixels .* gmax) |> colorview(Gray)
    
    save("t.jpg", gs)
    gs
end

write_raw (generic function with 1 method)

In [54]:
using Random 
pgs, as = progs(2,2)
es = exprs(pgs, as)
println(es)
a = string2args("EAGHNFODVNJCP"[2:end])

Random.seed!(1234)
a = rand(as)
aa = ones(as)

ps = polys(es, aa)

println(ps)

n = 1000000
iters = Dict{String, Vector{Float64}}()
for d in keys(ps)
    iters[d] = zeros(Float64, n)
end

states = Dict{String, Float64}()
states["x"] = 0.05
states["y"] = 0.05
states["z"] = 0.05
states["w"] = 0.05

for i in 1:n
    for d in keys(iters)
        states[d] = iters[d][i] = ps[d](x=>states["x"], y=>states["y"], z=>states["z"], w=>states["w"])
    end
end

write_raw(iters, 800, 600)


Dict{String, Expr}("x" => :(a[1] + a[2] * x + a[3] * x * x + a[4] * x * y + a[5] * y + a[6] * y * y), "y" => :(a[7] + a[8] * x + a[9] * x * x + a[10] * x * y + a[11] * y + a[12] * y * y))
Dict{String, Polynomial{true, Float64}}("x" => 0.5662374165061859x² + 0.4600853424625171xy + 0.8541465903790502y² + 0.7667970365022592x + 0.7940257103317943y + 0.5908446386657102, "y" => 0.24683718661000897x² + 0.5796722333690416xy + 0.010905889635595356y² + 0.2986142783434118x + 0.6488819502093455y + 0.20058603493384108)


LoadError: InexactError: trunc(Int64, NaN)